In [54]:
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import io
import os
from lightkurve import search_targetpixelfile
import  matplotlib.pyplot as plt
import lightkurve as lk
from tqdm import tqdm
import concurrent.futures




pd.set_option('display.max_rows', None)

%matplotlib inline

In [55]:

BASE_URL = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query="
QUERY = 'SELECT * FROM toi'
requestUrl = BASE_URL + QUERY.replace(' ', '+') + '&format=csv'
toiDf = pd.read_csv(requestUrl)
toiDf.head()



,tid,toi,toidisplay,toipfx,ctoi_alias,pl_pnum,tfopwg_disp,st_tmag,st_tmagerr1,st_tmagerr2,st_tmagsymerr,st_tmaglim,rastr,ra,raerr1,raerr2,rasymerr,decstr,dec,decerr1,decerr2,decsymerr,st_pmra,st_pmraerr1,st_pmraerr2,st_pmrasymerr,st_pmralim,st_pmdec,st_pmdecerr1,st_pmdecerr2,st_pmdecsymerr,st_pmdeclim,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidsymerr,pl_tranmidlim,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbpersymerr,pl_orbperlim,pl_trandurh,pl_trandurherr1,pl_trandurherr2,pl_trandurhsymerr,pl_trandurhlim,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandepsymerr,pl_trandeplim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radesymerr,pl_radelim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insolsymerr,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtsymerr,pl_eqtlim,st_dist,st_disterr1,st_disterr2,st_distsymerr,st_distlim,st_teff,st_tefferr1,st_tefferr2,st_teffsymerr,st_tefflim,st_logg,st_loggerr1,st_loggerr2,st_loggsymerr,st_logglim,st_rad,st_raderr1,st_raderr2,st_radsymerr,st_radlim,sectors,toi_created,rowupdate,release_date
0,79748331,1064.01,TOI-1064.01,1064,7.974833e+07,1,CP,10.0059,0.006,-0.006,1,0,19h44m00.94s,296.003935,NaN,NaN,NaN,-47d33m43.31s,-47.562030,NaN,NaN,NaN,-3.457,0.124,-0.124,1.0,0.0,-100.891,0.072,-0.072,1.0,0.0,2.458657e+06,0.001185,-0.001185,1,0,6.443866,0.000017,-0.000017,1,0,1.930945,0.301497,-0.301497,1,0,1121.951227,76.157646,-76.157646,1,0,2.460344,1.638728,-1.638728,1,0,60.831305,NaN,NaN,NaN,NaN,712.281284,NaN,NaN,NaN,NaN,68.0726,0.3620,-0.3620,1,0,4803.0,178.0,-178.0,1,0,4.52079,2.00348,-2.00348,1,0,0.737189,0.056703,-0.056703,1,0,NaN,2019-08-16 20:20:42,2024-04-25 10:08:01,2025-08-25 10:17:58
1,79748331,1064.02,TOI-1064.02,1064,7.974833e+07,2,CP,10.0059,0.006,-0.006,1,0,19h44m00.94s,296.003935,NaN,NaN,NaN,-47d33m43.31s,-47.562030,NaN,NaN,NaN,-3.457,0.124,-0.124,1.0,0.0,-100.891,0.072,-0.072,1.0,0.0,2.458664e+06,0.001696,-0.001696,1,0,12.226560,0.000027,-0.000027,1,0,2.190438,0.421978,-0.421978,1,0,1206.369329,86.609440,-86.609440,1,0,2.601477,1.753340,-1.753340,1,0,25.897038,NaN,NaN,NaN,NaN,575.350205,NaN,NaN,NaN,NaN,68.0726,0.3620,-0.3620,1,0,4803.0,178.0,-178.0,1,0,4.52079,2.00348,-2.00348,1,0,0.737189,0.056703,-0.056703,1,0,NaN,2019-08-16 20:20:42,2024-04-25 10:08:01,2025-08-25 10:17:58
2,7088246,1065.01,TOI-1065.01,1065,7.088246e+06,1,KP,13.2851,0.006,-0.006,1,0,19h05m28.02s,286.366750,NaN,NaN,NaN,-50d04m02.46s,-50.067350,NaN,NaN,NaN,7.315,0.037,-0.037,1.0,0.0,-0.862,0.029,-0.029,1.0,0.0,2.458654e+06,0.002094,-0.002094,1,0,2.160344,0.000276,-0.000276,1,0,1.828854,0.144810,-0.144810,1,0,19357.992992,1487.618900,-1487.618900,1,0,22.503541,17.500204,-17.500204,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,639.0040,9.4715,-9.4715,1,0,5780.0,NaN,NaN,1,0,4.32928,NaN,NaN,1,0,1.200000,NaN,NaN,1,0,NaN,2019-08-16 20:20:41,2021-10-29 12:59:15,2025-08-25 10:17:58
3,201604954,1066.01,TOI-1066.01,1066,2.016050e+08,1,KP,11.9707,0.006,-0.006,1,0,19h00m23.16s,285.096517,NaN,NaN,NaN,-54d53m36.03s,-54.893341,NaN,NaN,NaN,3.990,0.054,-0.054,1.0,0.0,-37.190,0.042,-0.042,1.0,0.0,2.458658e+06,0.001239,-0.001239,1,0,4.606184,0.000392,-0.000392,1,0,3.121643,0.078877,-0.078877,1,0,13793.034867,1031.553200,-1031.553200,1,0,12.962219,0.582183,-0.582183,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,353.8910,4.9190,-4.9190,1,0,5670.0,NaN,NaN,1,0,4.39250,NaN,NaN,1,0,1.070000,NaN,NaN,1,0,NaN,2019-08-16 20:20:44,2021-10-29 12:59:15,2025-08-25 10:17:58
4,201642601,1067.01,TOI-1067.01,1067,2.016426e+08,1,KP,13.1686,0.006,-0.006,1,0,19h14m41.28s,288.672005,NaN,NaN,NaN,-59d34m45.66s,-59.579350,NaN,NaN,NaN,3.125,0.031,-0.031,1.0,0.0,6.146,0.029,-0.029,1.0,0.0,2.458657e+06,0.000501,-0.000501,1,0,3.131674,0.000003,-0.000003,1,0,2.457920,0.081564,-0.081564,1,0,14915.694372,307.913640,-307.913640,1,0,9.499519,0.344233,-0.344233,1,0,148.654499,NaN,NaN,NaN,NaN,890.562213,NaN,NaN,NaN,NaN,263.6090,1.6590,-1.6590,1,0,4351.0,167.0,-167.0,1,0,4.43800,NaN,NaN,1,0,0.767010,NaN,NaN,1,0,NaN,2019-08-16 20:20:44,2024-03-14 16:54:32,2025-08-25 10:17:58


In [56]:


TRUE_PLANETS = ['CP', 'KP', 'PC']

FALSE_POSITIVES = ['FP', 'FA']


toi_filtered = toiDf[toiDf['tfopwg_disp'].isin(TRUE_PLANETS + FALSE_POSITIVES)].copy()
toi_filtered['isPlanet'] = 0

toi_filtered.loc[toi_filtered['tfopwg_disp'].isin(TRUE_PLANETS), 'isPlanet'] = 1

toi_filtered['isPlanet'].value_counts()

missing_values = toi_filtered.isnull().sum()
missing_percentage = (missing_values / len(toi_filtered)) * 100


# remove all columns that have 100% missing values
toi_filtered = toi_filtered.loc[:, missing_percentage < 100]


missing_values = toi_filtered.isnull().sum()
missing_percentage = (missing_values / len(toi_filtered)) * 100

print(missing_percentage)


tid                   0.000000
toi                   0.000000
toidisplay            0.000000
toipfx                0.000000
ctoi_alias            0.000000
pl_pnum               0.000000
tfopwg_disp           0.000000
st_tmag               0.000000
st_tmagerr1           0.000000
st_tmagerr2           0.000000
st_tmagsymerr         0.000000
st_tmaglim            0.000000
rastr                 0.000000
ra                    0.000000
decstr                0.000000
dec                   0.000000
st_pmra               1.708571
st_pmraerr1           1.708571
st_pmraerr2           1.708571
st_pmrasymerr         1.708571
st_pmralim            1.708571
st_pmdec              1.708571
st_pmdecerr1          1.708571
st_pmdecerr2          1.708571
st_pmdecsymerr        1.708571
st_pmdeclim           1.708571
pl_tranmid            0.000000
pl_tranmiderr1        0.152799
pl_tranmiderr2        0.152799
pl_tranmidsymerr      0.000000
pl_tranmidlim         0.000000
pl_orbper             1.402973
pl_orbpe

In [57]:
cols_to_remove = ['pl_radeerr1', 'pl_radeerr2','st_disterr1', 'st_disterr2' ,'st_loggerr1', 'st_loggerr2', 'st_raderr1', 'st_raderr2']
toi_removed = toi_filtered.drop(columns=cols_to_remove)

missing_values = toi_removed.isnull().sum()
missing_percentage = (missing_values / len(toi_removed)) * 100
print(missing_percentage)

tid                   0.000000
toi                   0.000000
toidisplay            0.000000
toipfx                0.000000
ctoi_alias            0.000000
pl_pnum               0.000000
tfopwg_disp           0.000000
st_tmag               0.000000
st_tmagerr1           0.000000
st_tmagerr2           0.000000
st_tmagsymerr         0.000000
st_tmaglim            0.000000
rastr                 0.000000
ra                    0.000000
decstr                0.000000
dec                   0.000000
st_pmra               1.708571
st_pmraerr1           1.708571
st_pmraerr2           1.708571
st_pmrasymerr         1.708571
st_pmralim            1.708571
st_pmdec              1.708571
st_pmdecerr1          1.708571
st_pmdecerr2          1.708571
st_pmdecsymerr        1.708571
st_pmdeclim           1.708571
pl_tranmid            0.000000
pl_tranmiderr1        0.152799
pl_tranmiderr2        0.152799
pl_tranmidsymerr      0.000000
pl_tranmidlim         0.000000
pl_orbper             1.402973
pl_orbpe

In [58]:

print(f"Catálogo carregado. Total de {len(toi_filtered)} sinais para processar.")
print(f"Exemplos positivos (isPlanet=1): {len(toi_filtered[toi_filtered['isPlanet'] == 1])}")
print(f"Exemplos negativos (isPlanet=0): {len(toi_filtered[toi_filtered['isPlanet'] == 0])}")

Catálogo carregado. Total de 7199 sinais para processar.
Exemplos positivos (isPlanet=1): 5914
Exemplos negativos (isPlanet=0): 1285


In [74]:
toi_null_removed = toi_removed.dropna().copy()
print(f"Após remoção de nulos, total de {len(toi_null_removed)} sinais para processar.")

Após remoção de nulos, total de 6021 sinais para processar.


In [75]:
# Numero de estrelas diferentes

toi_null_removed['star_id'] = toi_null_removed['toi'].apply(lambda x: str(x)[-2:])

print(toi_null_removed['toi'].apply(lambda x: str(x)[-2:]).value_counts())
print(f"Número de estrelas diferentes: {toi_null_removed['star_id'].nunique()}")

toi
01    5763
02     202
03      44
04       9
05       3
Name: count, dtype: int64
Número de estrelas diferentes: 5


In [76]:
cols_to_drop = [
    'tid', 'toi', 'toidisplay', 'toipfx', 'ctoi_alias', 'pl_pnum',
    'tfopwg_disp', 'rastr', 'decstr', 'toi_created', 'rowupdate', 'release_date',
    'st_tmagsymerr', 'st_tmaglim', 'st_pmrasymerr', 'st_pmralim',
    'st_pmdecsymerr', 'st_pmdeclim', 'pl_tranmidsymerr', 'pl_tranmidlim',
    'pl_orbpersymerr', 'pl_orbperlim', 'pl_trandurhsymerr', 'pl_trandurhlim',
    'pl_trandepsymerr', 'pl_trandeplim', 'pl_radesymerr', 'pl_radelim',
    'st_distsymerr', 'st_distlim', 'st_teffsymerr', 'st_tefflim',
    'st_loggsymerr', 'st_logglim', 'st_radsymerr', 'st_radlim'
]

# Removendo apenas as colunas que existem no DataFrame
cols_to_drop_existing = [col for col in cols_to_drop if col in toi_null_removed.columns]
data_for_model = toi_null_removed.drop(columns=cols_to_drop_existing)

print("\nColunas após a remoção:", data_for_model.shape[1])
print("Features que serão usadas no modelo:")
print(data_for_model.columns.tolist())


Colunas após a remoção: 34
Features que serão usadas no modelo:
['st_tmag', 'st_tmagerr1', 'st_tmagerr2', 'ra', 'dec', 'st_pmra', 'st_pmraerr1', 'st_pmraerr2', 'st_pmdec', 'st_pmdecerr1', 'st_pmdecerr2', 'pl_tranmid', 'pl_tranmiderr1', 'pl_tranmiderr2', 'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_trandurh', 'pl_trandurherr1', 'pl_trandurherr2', 'pl_trandep', 'pl_trandeperr1', 'pl_trandeperr2', 'pl_rade', 'pl_insol', 'pl_eqt', 'st_dist', 'st_teff', 'st_tefferr1', 'st_tefferr2', 'st_logg', 'st_rad', 'isPlanet', 'star_id']


In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = data_for_model.drop(columns=['isPlanet', 'star_id'])
y = data_for_model['isPlanet']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

groups = data_for_model['star_id']
print(f"Total de sinais: {len(X)}")
print(f"Total de estrelas únicas (grupos): {len(groups.unique())}")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Tamanho do conjunto de treino: {X_train_scaled.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test_scaled.shape[0]} amostras")


# Relação entre classes no conjunto de treino
print("Distribuição das classes no conjunto de treino:")
print(y_train.value_counts(normalize=True))

Total de sinais: 6021
Total de estrelas únicas (grupos): 5
Tamanho do conjunto de treino: 4816 amostras
Tamanho do conjunto de teste: 1205 amostras
Distribuição das classes no conjunto de treino:
isPlanet
1    0.837209
0    0.162791
Name: proportion, dtype: float64


In [ ]:
# %%
# Importando os modelos e as métricas
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import (
	accuracy_score,
	classification_report,
	roc_auc_score,
	average_precision_score
)

count_neg = y_train.value_counts()[0]
count_pos = y_train.value_counts()[1]
scale_pos_weight = count_neg / count_pos

models = {
	"Regressão Logística": LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
	"LDA": LinearDiscriminantAnalysis(),
	"Árvore de Decisão": DecisionTreeClassifier(random_state=42, class_weight='balanced'),
	"Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
	"SVM": SVC(random_state=42, probability=True, class_weight='balanced'),
	"XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight),
	"XGBoost (Equilibrado)": XGBClassifier(
		random_state=42,
		use_label_encoder=False,
		eval_metric='logloss',
		scale_pos_weight=scale_pos_weight,
		n_estimators=600,
		learning_rate=0.04,
		max_depth=6,
		subsample=0.8,
		colsample_bytree=0.8
	),
	"LightGBM": LGBMClassifier(random_state=42, class_weight='balanced'),
	"LightGBM (Tunado v3)": LGBMClassifier(
		random_state=42,
		class_weight='balanced',
		n_estimators=500,
		learning_rate=0.05,
	),
}


Treinando o modelo: Regressão Logística...
Treinando o modelo: LDA...
Treinando o modelo: Árvore de Decisão...
Treinando o modelo: Random Forest...
Treinando o modelo: SVM...
Treinando o modelo: XGBoost...


c:\Users\Guilherme\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:09:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Treinando o modelo: XGBoost (Equilibrado)...


c:\Users\Guilherme\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:09:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Treinando o modelo: LightGBM...
[LightGBM] [Info] Number of positive: 4032, number of negative: 784
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000729 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7232
[LightGBM] [Info] Number of data points in the train set: 4816, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


c:\Users\Guilherme\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Guilherme\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Treinando o modelo: LightGBM (Tunado v3)...
[LightGBM] [Info] Number of positive: 4032, number of negative: 784
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000432 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7232
[LightGBM] [Info] Number of data points in the train set: 4816, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000

Todos os modelos foram treinados e avaliados.


c:\Users\Guilherme\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Guilherme\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:

# Lista para armazenar os resultados de cada modelo
results = []

# Loop para treinar e avaliar cada modelo
for name, model in models.items():
	print(f"Treinando o modelo: {name}...")

	# Treinar o modelo
	model.fit(X_train_scaled, y_train)

	# Fazer previsões no conjunto de teste
	y_pred = model.predict(X_test_scaled)
	y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] # Probabilidades para a classe positiva (1)

	# Calcular as métricas
	accuracy = accuracy_score(y_test, y_pred)
	accuracyNonPlanet = accuracy_score(y_test, 1 - y_pred)
	report = classification_report(y_test, y_pred, output_dict=True)
	roc_auc = roc_auc_score(y_test, y_pred_proba)
	prc_auc = average_precision_score(y_test, y_pred_proba)

	# Armazenar os resultados
	results.append({
		"Modelo": name,
		"AUC ROC": roc_auc,
		"AUC PRC": prc_auc,
		"Acurácia": accuracy,
		"Acurácia (FP)": accuracyNonPlanet,
		"Precisão (Planeta)": report['1']['precision'],
		"Precisão (FP)": report['0']['precision'],
		"Recall (Planeta)": report['1']['recall'],
		"Recall (FP)": report['0']['recall'],
		"F1-Score (Planeta)": report['1']['f1-score'],
		"F1-Score (FP)": report['0']['f1-score'],
		
	})

print("\nTodos os modelos foram treinados e avaliados.")

In [65]:
# %%
# Criar um DataFrame com os resultados
results_df = pd.DataFrame(results)

# Ordenar o DataFrame pelo AUC ROC para ver os melhores modelos primeiro
results_df_sorted = results_df.sort_values(by="AUC ROC", ascending=False)

# Exibir a tabela de resultados formatada
display(results_df_sorted.style.format({
	'Acurácia': '{:.4f}',
	'Acurácia (FP)': '{:.4f}',
	'Precisão (Planeta)': '{:.4f}',
	'Recall (Planeta)': '{:.4f}',
	'Precisão (FP)': '{:.4f}',
	'Recall (FP)': '{:.4f}',
	'F1-Score (Planeta)': '{:.4f}',
	'F1-Score (FP)': '{:.4f}',
	'AUC ROC': '{:.4f}',
	'AUC PRC': '{:.4f}'
}).background_gradient(cmap='viridis', subset=['Acurácia', 'F1-Score (Planeta)', 'AUC ROC', 'AUC PRC']))

,Modelo,AUC ROC,AUC PRC,Acurácia,Acurácia (FP),Precisão (Planeta),Precisão (FP),Recall (Planeta),Recall (FP),F1-Score (Planeta),F1-Score (FP)
6,XGBoost (Equilibrado),0.8820,0.9679,0.8780,0.1220,0.9193,0.6384,0.9366,0.5765,0.9278,0.6059
8,LightGBM (Tunado v3),0.8735,0.9652,0.8747,0.1253,0.9078,0.6471,0.9465,0.5051,0.9267,0.5673
5,XGBoost,0.8708,0.9653,0.8664,0.1336,0.9117,0.6000,0.9306,0.5357,0.9210,0.5660
7,LightGBM,0.8612,0.9612,0.8705,0.1295,0.9210,0.6042,0.9247,0.5918,0.9228,0.5979
4,SVM,0.8604,0.9610,0.8241,0.1759,0.9394,0.4732,0.8444,0.7194,0.8894,0.5709
3,Random Forest,0.8582,0.9615,0.8622,0.1378,0.8733,0.6974,0.9772,0.2704,0.9224,0.3897
0,Regressão Logística,0.8161,0.9464,0.7701,0.2299,0.9347,0.3884,0.7800,0.7194,0.8504,0.5045
1,LDA,0.8149,0.9485,0.8622,0.1378,0.8808,0.6531,0.9663,0.3265,0.9216,0.4354
2,Árvore de Decisão,0.6576,0.8829,0.8224,0.1776,0.8878,0.4500,0.9019,0.4133,0.8948,0.4309
